## Creating confidence intervals in python
In this assessment, you will look at data from a study on toddler sleep habits. 

The confidence intervals you create and the questions you answer in this Jupyter notebook will be used to answer questions in the following graded assignment.

In [3]:
import numpy as np
import pandas as pd
from scipy.stats import t
pd.set_option('display.max_columns', 30) # set so can see all columns of the DataFrame

Your goal is to analyse data which is the result of a study that examined
differences in a number of sleep variables between napping and non-napping toddlers. Some of these
sleep variables included: Bedtime (lights-off time in decimalized time), Night Sleep Onset Time (in
decimalized time), Wake Time (sleep end time in decimalized time), Night Sleep Duration (interval
between sleep onset and sleep end in minutes), and Total 24-Hour Sleep Duration (in minutes). Note:
[Decimalized time](https://en.wikipedia.org/wiki/Decimal_time) is the representation of the time of day using units which are decimally related.   


The 20 study participants were healthy, normally developing toddlers with no sleep or behavioral
problems. These children were categorized as napping or non-napping based upon parental report of
children’s habitual sleep patterns. Researchers then verified napping status with data from actigraphy (a
non-invasive method of monitoring human rest/activity cycles by wearing of a sensor on the wrist) and
sleep diaries during the 5 days before the study assessments were made.


You are specifically interested in the results for the Bedtime, Night Sleep Duration, and Total 24-
Hour Sleep Duration. 


Reference: Akacem LD, Simpkin CT, Carskadon MA, Wright KP Jr, Jenni OG, Achermann P, et al. (2015) The Timing of the Circadian Clock and Sleep Differ between Napping and Non-Napping Toddlers. PLoS ONE 10(4): e0125181. https://doi.org/10.1371/journal.pone.0125181

In [4]:
df = pd.read_excel('../nap-nonap/S1_Dataset.xlsx')

In [5]:
df

,id,sex,age (months),dlmo time,days napped,napping,nap lights outl time,nap sleep onset,nap midsleep,nap sleep offset,nap wake time,nap duration,nap time in bed,night bedtime,night sleep onset,sleep onset latency,night midsleep time,night wake time,night sleep duration,night time in bed,24 h sleep duration,bedtime phase difference,sleep onset phase difference,midsleep phase difference,wake time phase difference
0,1,female,33.7,19.24,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.45,20.68,0.23,1.92,7.17,629.40,643.00,629.40,-1.21,-1.44,6.68,11.93
1,2,female,31.5,18.27,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.23,19.48,0.25,1.09,6.69,672.40,700.40,672.40,-0.96,-1.21,6.82,12.42
2,3,male,31.9,19.14,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.60,20.05,0.45,1.29,6.53,628.80,682.60,628.80,-0.46,-0.91,6.15,11.39
3,4,female,31.6,19.69,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.46,19.50,0.05,1.89,8.28,766.60,784.00,766.60,0.23,0.19,6.20,12.59
4,5,female,33.0,19.52,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.21,19.65,0.45,1.30,6.95,678.00,718.00,678.00,0.31,-0.13,5.78,11.43
5,6,female,36.2,18.22,4.0,1.0,14.00,14.22,15.00,15.78,16.28,93.75,137.00,19.95,20.25,0.29,1.26,6.28,602.20,653.80,695.95,-1.73,-2.03,7.05,12.06
6,7,male,36.3,19.28,1.0,1.0,14.75,15.03,15.92,16.80,16.08,106.00,80.00,20.60,20.96,0.36,2.12,7.27,618.40,655.40,724.40,-1.32,-1.68,6.84,11.99
7,8,male,30.0,21.06,5.0,1.0,13.09,13.43,14.44,15.46,15.82,121.60,163.80,22.01,22.53,0.51,2.92,7.31,526.80,582.40,648.40,-0.95,-1.47,5.86,10.25
8,9,male,33.2,19.38,2.0,1.0,14.41,14.42,15.71,17.01,16.60,155.50,131.25,20.24,20.37,0.13,1.60,6.82,626.80,660.33,782.30,-0.86,-0.99,6.22,11.44
9,10,female,37.1,19.93,3.0,1.0,13.12,13.42,14.31,15.19,15.30,106.67,130.67,20.78,21.63,0.84,2.20,6.52,549.50,626.00,656.17,-0.76,-1.82,6.21,10.59


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            22 non-null     object 
 1   sex                           20 non-null     object 
 2   age (months)                  20 non-null     float64
 3   dlmo time                     20 non-null     float64
 4   days napped                   20 non-null     float64
 5   napping                       20 non-null     float64
 6   nap lights outl time          15 non-null     float64
 7   nap sleep onset               15 non-null     float64
 8   nap midsleep                  15 non-null     float64
 9   nap sleep offset              15 non-null     float64
 10  nap wake time                 15 non-null     float64
 11  nap duration                  15 non-null     float64
 12  nap time in bed               15 non-null     float64
 13  night b

**Question**: What variable is used in the column 'napping' to indicate a toddler takes a nap? <br>
1

**Question**: What is the sample size $n$? <br>
20

## Hypothesis testing
We will look at two hypothesis test, each with $\alpha = .025$:  


1. Is the average bedtime for toddlers who nap later than the average bedtime for toddlers who don't nap?


$$H_0: \mu_{nap}=\mu_{no\ nap}, \ H_a:\mu_{nap}>\mu_{no\ nap}$$
Or equivalently:
$$H_0: \mu_{nap}-\mu_{no\ nap}=0, \ H_a:\mu_{nap}-\mu_{no\ nap}>0$$


2. The average 24 h sleep duration (in minutes) for napping toddlers is different from toddlers who don't nap.


$$H_0: \mu_{nap}=\mu_{no\ nap}, \ H_a:\mu_{nap}\neq\mu_{no\ nap}$$
Or equivalenty:
$$H_0: \mu_{nap}-\mu_{no\ nap}=0, \ H_a:\mu_{nap}-\mu_{no\ nap} \neq 0$$

Aside: This $\alpha$ level is equivalent to $\alpha = .05$ and then applying the [Bonferonni correction](https://en.wikipedia.org/wiki/Bonferroni_correction).

Before any analysis, we will convert 'night bedtime' into decimalized time. 

In [7]:
# Convert 'night bedtime' into decimalized time
df.loc[:,'night bedtime'] = np.floor(df['night bedtime'])*60 + np.round(df['night bedtime']%1,2 )*100

Now, isolate the column 'night bedtime' for those who nap into a new variable, and those who didn't nap into another new variable. 

In [8]:
nap_bedtime = df.loc[ df['napping'] == 1, 'night bedtime']


In [9]:
no_nap_bedtime = df.loc[ df['napping'] == 0, 'night bedtime']

Now find the sample mean bedtime for nap and no_nap.

In [10]:
nap_mean_bedtime = nap_bedtime.mean()
nap_mean_bedtime

1233.0666666666666

In [11]:
no_nap_mean_bedtime = no_nap_bedtime.mean()
no_nap_mean_bedtime

1191.0

In [12]:
nap_mean_bedtime - no_nap_mean_bedtime

42.066666666666606

**Question**: What is the sample difference of mean bedtime for nappers minus no nappers? <br>
Now find the sample standard deviation for $X_{nap}$ and $X_{no\ nap}$.

In [13]:
nap_s_bedtime = nap_bedtime.std()
nap_s_bedtime

34.445540177143954

In [15]:
s12 = nap_s_bedtime**2
s12

1186.4952380952384

In [14]:
no_nap_s_bedtime = no_nap_bedtime.std()
no_nap_s_bedtime

34.30014577228499

In [16]:
s22 = no_nap_s_bedtime**2
s22

1176.5

In [17]:
n1 = 15
n2 = 5

In [18]:
numerator = (n1-1)*s12 + (n2-1)*s22
numerator

21316.933333333338

In [19]:
denominator = n1+n2-2
denominator

18

In [20]:
element2 = (1/n1)+(1/n2)
element2

0.26666666666666666

**Question**: What is the s.e.$(\bar{X}_{nap} - \bar{X}_{no\ nap})$?

We expect the variance in sleep time for toddlers who nap and toddlers who don't nap to be the same. So we use a pooled standard error.

Calculate the pooled standard error of $\bar{X}_{nap} - \bar{X}_{no\ nap}$ using the formula below.

$s.e.(\bar{X}_{nap} - \bar{X}_{no\ nap}) = \sqrt{\frac{(n_1-1)s_1^2 + (n_2-1)s_2^2}{n_1+n_2-2}(\frac{1}{n_1}+\frac{1}{n_2})}$

In [21]:
import statsmodels.api as sm

napper = df.loc[df.napping==1, "night bedtime"].dropna()
napper = sm.stats.DescrStatsW(napper)
nonapper = df.loc[df.napping==0, "night bedtime"].dropna()
nonapper = sm.stats.DescrStatsW(nonapper)
print("pooled: Ztest  ", sm.stats.CompareMeans(napper, nonapper).ztest_ind(usevar='pooled'))
print("unequal: Ztest  ", sm.stats.CompareMeans(napper, nonapper).ztest_ind(usevar='unequal'))
print("pooled: ttest_imd  ", sm.stats.CompareMeans(napper, nonapper).ttest_ind(usevar='pooled', alternative='larger'))
print("unequal: ttest_ind  ", sm.stats.CompareMeans(napper, nonapper).ttest_ind(usevar='unequal'))


pooled: Ztest   (2.367160052079275, 0.017925179082180703)
unequal: Ztest   (2.372449904068502, 0.017670559204472077)
pooled: ttest_imd   (2.367160052079275, 0.014667451430902697, 18.0)
unequal: ttest_ind   (2.372449904068502, 0.04984441347729212, 6.917984223110409)


In [22]:
pooled_se = np.sqrt((numerator / denominator) * element2)
pooled_se

17.77094313065816

**Question**: Given our sample size of $n$, how many degrees of freedom ($df$) are there for the associated $t$ distribution?

In [23]:
19

19

Now calculate the $t$-test statistic for our first hypothesis test using  
* pooled s.e.($\bar{X}_{nap} - \bar{X}_{no\ nap}$)  
* $\bar{X}_{nap} - \bar{X}_{no\ nap}$  
* $\mu_{0,\ nap} - \mu_{0,\ no\ nap}=0$, the population difference in means under the null hypothesis

In [24]:
test_statistic = (nap_mean_bedtime - no_nap_mean_bedtime - 0) / pooled_se
test_statistic

2.367160052079275

**Question**: What is the p-value for the first hypothesis test?

To find the p-value, we can use the function:
```
t.cdf(y, df)
```
Which for $X \sim t(df)$ returns $P(X \leq y)$.

Because of the symmetry of the $t$ distrubution, we have that 
```
1-t.cdf(y, df)
```
returns $P(X > y)$

The function t.cdf(y, df) will give you the same value as finding the one-tailed probability of y on a t-table with the specified degrees of freedom.

Use the function t.cdf(y, df) to find the p-value for the first hypothesis test.

In [25]:
import scipy.stats.distributions as dist
pvalue = dist.norm.cdf(-np.abs(test_statistic))
pvalue #Single sided

0.008962589541090351

**Question**: What are the t-statistic and p-value for the second hypothesis test?

Calculate the $t$ test statistics and corresponding p-value using the scipy function scipy.stats.ttest_ind(a, b, equal_var=True) and check with your answer. 

**Question**: Does scipy.stats.ttest_ind return values for a one-sided or two-sided test? 2

**Question**: Can you think of a way to recover the results you got using 1-t.cdf from the p-value given by scipy.stats.ttest_ind?

Use the scipy function scipy.stats.ttest_ind(a, b, equal_var=True) to find the $t$ test statistic and corresponding p-value for the second hypothesis test.

In [26]:
import scipy
from scipy import stats
x = scipy.stats.ttest_ind(df.loc[df.napping==1, "night bedtime"].dropna(), df.loc[df.napping==0, "night bedtime"].dropna(), equal_var=True)
x

Ttest_indResult(statistic=2.367160052079275, pvalue=0.029334902861805394)

In [28]:
print(x[1], '!=')
print(x[1]/2, '> or <')

0.029334902861805394 !=
0.014667451430902697 > or <


**Question**: For the $\alpha=.025$, do you reject or fail to reject the first hypothesis?
reject

**Question**: For the $\alpha=.025$, do you reject or fail to reject the second hypothesis?
fail to reject